In [2]:
#!/usr/bin/env python
# coding: utf-8

# In[5]:


# Data Transformation 
import numpy as np
import pandas as pd
def Transform(train_slices, test_slices):
    def process_transform_dataset(df):
        samples = []
        m = 298
        k = 3
        n = len(df.index)
        for i in range(n):
            x = df.iloc[i].tolist()
            sample = []
            for j in range(m):
                index = j 
                for p in range(k):
                    sample.append(x[index])
                    index = index + m
            sample.append(x[len(x)-1])
            samples.append(sample)
        return samples

    train_df = pd.DataFrame(train_slices)
    test_df = pd.DataFrame(test_slices)


    train_transform = process_transform_dataset(train_df)
    test_transform = process_transform_dataset(test_df)
    return train_transform, test_transform
    


# In[ ]:

In [ ]:
import os
import numpy as np
import pandas as pd
import time
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn import metrics
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt

# Set random seeds for reproducibility
np.random.seed(1)
import random as rn
rn.seed(1)
import tensorflow as tf
tf.random.set_seed(1)

# Load data
train = pd.read_csv(os.path.join(os.getcwd(),'Input', 'AsphaltObstacle_Train.csv'))
test = pd.read_csv(os.path.join(os.getcwd(),'Input', 'AsphaltObstacle_Test.csv'))

# Trasformation
train_transform, test_transform = Transform(train, test)
train = pd.DataFrame(train_transform)
test = pd.DataFrame(test_transform)

# Preprocess the data
x_train, y_train = train.iloc[:, :-1].values, train.iloc[:, -1].values
x_test, y_test = test.iloc[:, :-1].values, test.iloc[:, -1].values


# Feature scaling
sc = StandardScaler()
x_train = sc.fit_transform(x_train).reshape(x_train.shape[0], 298, 3)
x_test = sc.transform(x_test).reshape(x_test.shape[0], 298, 3)

# Train the model
from sklearn.utils.class_weight import compute_class_weight

# Compute class weights for each class
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)

# Create a dictionary that maps class labels to their corresponding weight
class_weight_dict = {i: weight for i, weight in enumerate(class_weights)}

# One-hot encode labels
encoder = OneHotEncoder(sparse_output=False)
y_train = encoder.fit_transform(y_train.reshape(-1, 1))
y_test = encoder.transform(y_test.reshape(-1, 1))

# Set up directories and model parameters
base_directory = os.path.join(os.getcwd(), 'Output', 'LSTM')
os.makedirs(base_directory, exist_ok=True)

# Define model parameters and train
m, epochs, patience = 24,200,4  # example values
subdirectory = os.path.join(base_directory, f"{m}.{m}.{epochs}.{patience}")
os.makedirs(subdirectory, exist_ok=True)

# Build the model
model = Sequential([
    LSTM(m, return_sequences=True, input_shape=(298, 3)),
    LSTM(m),
    Dense(4, activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Callbacks
es = EarlyStopping(monitor='val_loss', mode='min', patience=patience)
cp = ModelCheckpoint(os.path.join(subdirectory, 'bestweights.keras'), monitor='val_loss', mode='min', save_best_only=True)


# Train the model with class weights
model.fit(x_train, y_train, validation_split=0.1, epochs=epochs, batch_size=12, 
          callbacks=[es, cp], class_weight=class_weight_dict)


# Evaluate the model
scores = model.evaluate(x_test, y_test, verbose=0)
print(f"Test Accuracy: {scores[1] * 100:.2f}%")

# Predict and calculate testing time
start_time = time.time()
y_pred = model.predict(x_test)
end_time = time.time()
testing_time = end_time - start_time
print(f"Testing Time: {testing_time} seconds")

# Save testing time
with open(os.path.join(subdirectory, 'testing_time.txt'), "w") as f:
    f.write(f"Testing Time: {testing_time} seconds")

# Save class probabilities
np.savetxt(os.path.join(subdirectory, 'class_probabilities.csv'), y_pred, delimiter=',')

# Classification report and confusion matrix
y_pred_labels = np.argmax(y_pred, axis=1)
y_test_labels = np.argmax(y_test, axis=1)

# Save classification report
report_str = classification_report(y_test_labels, y_pred_labels)
with open(os.path.join(subdirectory, 'classification_report.txt'), "w") as f:
    f.write(report_str)

# Confusion matrix plot
disp = metrics.ConfusionMatrixDisplay.from_predictions(y_test_labels, y_pred_labels)
disp.figure_.suptitle("Confusion Matrix")
disp.figure_.savefig(os.path.join(subdirectory, 'Confusion_Matrix.png'))
plt.close()


Epoch 1/200


C:\Users\tenzi\anaconda3\envs\aeon-env\lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


30/30 ━━━━━━━━━━━━━━━━━━━━ 14s 288ms/step - accuracy: 0.2271 - loss: 1.3997 - val_accuracy: 0.0256 - val_loss: 1.4610
Epoch 2/200
30/30 ━━━━━━━━━━━━━━━━━━━━ 10s 268ms/step - accuracy: 0.2920 - loss: 1.3583 - val_accuracy: 0.3333 - val_loss: 1.4073
Epoch 3/200
30/30 ━━━━━━━━━━━━━━━━━━━━ 7s 241ms/step - accuracy: 0.3528 - loss: 1.2623 - val_accuracy: 0.5641 - val_loss: 1.2029
Epoch 4/200
30/30 ━━━━━━━━━━━━━━━━━━━━ 10s 247ms/step - accuracy: 0.4126 - loss: 1.2093 - val_accuracy: 0.6667 - val_loss: 1.0445
Epoch 5/200
30/30 ━━━━━━━━━━━━━━━━━━━━ 10s 241ms/step - accuracy: 0.4256 - loss: 1.1737 - val_accuracy: 0.7949 - val_loss: 0.8640
Epoch 6/200
30/30 ━━━━━━━━━━━━━━━━━━━━ 7s 246ms/step - accuracy: 0.4482 - loss: 1.1382 - val_accuracy: 0.8462 - val_loss: 0.9166
Epoch 7/200
30/30 ━━━━━━━━━━━━━━━━━━━━ 6s 212ms/step - accuracy: 0.4724 - loss: 1.1091 - val_accuracy: 0.8462 - val_loss: 0.8576
Epoch 8/200
30/30 ━━━━━━━━━━━━━━━━━━━━ 6s 211ms/step - accuracy: 0.4761 - loss: 1.1211 - val_accuracy: 0.